In [1]:
1 + 1

2

In [ ]:
# スピン値(配列で表示)
σ = [-1, 1]

# ハミルトニアンの計算（隣接ペアの和）
function energy(config, L, J)　#config:スピン配置を1次元配列として渡す
    E = 0.0 #全体のエネルギーを初期化
    for x in 1:L
        for y in 1:L
            s = config[(x-1)*L + y]
            # 右隣（周期境界なし）
            if y < L
                s_r = config[(x-1)*L + y + 1]
                E -= J * s * s_r
            end
            # 下隣
            if x < L
                s_d = config[x*L + y]
                E -= J * s * s_d
            end
        end
    end
    return E
end

# 分配関数を計算
function partition_function(L; β=1.0, J=1.0)
    Z = 0.0
    N = L * L
    # 全スピン状態列挙（2^(L^2)）
    for i in 0:(2^N - 1)
        config = zeros(Int, N)
        for k in 1:N
            # ビットを使ってスピン割り当て
            config[k] = σ[bitstring(i)[end-k+1] == '1' ? 2 : 1]
        end
        E = energy(config, L, J)
        Z += exp(-β * E)
    end
    return Z
end

# 例：L=2, β=0.5, J=1.0
println("Z =", partition_function(2, β=0.5, J=1.0))

Z =27.048782764334526


In [3]:
using Pkg
Pkg.add("ITensors")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [5]:
#4脚テンソル
using ITensors

s1 = Index(2, "s1,a")
s2 = Index(2, "s2,b")
s3 = Index(2, "s3,c")
s4 = Index(2, "s4,d")

@show T4 = ITensor(s1, s2, s3, s4) #ここで4脚テンソルの関数を作成

T4 = ITensor(s1, s2, s3, s4) = ITensor ord=4
Dim 1: (dim=2|id=868|"a,s1")
Dim 2: (dim=2|id=673|"b,s2")
Dim 3: (dim=2|id=615|"c,s3")
Dim 4: (dim=2|id=761|"d,s4")
NDTensors.EmptyStorage{NDTensors.EmptyNumber, NDTensors.Dense{NDTensors.EmptyNumber, Vector{NDTensors.EmptyNumber}}}
 2×2×2×2




ITensor ord=4 (dim=2|id=868|"a,s1") (dim=2|id=673|"b,s2") (dim=2|id=615|"c,s3") (dim=2|id=761|"d,s4")
NDTensors.EmptyStorage{NDTensors.EmptyNumber, NDTensors.Dense{NDTensors.EmptyNumber, Vector{NDTensors.EmptyNumber}}}

In [6]:
#3脚テンソル
s1 = Index(2, "s1,a")
s2 = Index(2, "s2,b")
s3 = Index(2, "s3,c")

@show T3 = ITensor(s1, s2, s3)

T3 = ITensor(s1, s2, s3) = ITensor ord=3
Dim 1: (dim=2|id=572|"a,s1")
Dim 2: (dim=2|id=804|"b,s2")
Dim 3: (dim=2|id=86|"c,s3")
NDTensors.EmptyStorage{NDTensors.EmptyNumber, NDTensors.Dense{NDTensors.EmptyNumber, Vector{NDTensors.EmptyNumber}}}
 2×2×2




ITensor ord=3 (dim=2|id=572|"a,s1") (dim=2|id=804|"b,s2") (dim=2|id=86|"c,s3")
NDTensors.EmptyStorage{NDTensors.EmptyNumber, NDTensors.Dense{NDTensors.EmptyNumber, Vector{NDTensors.EmptyNumber}}}

In [7]:
#2脚テンソル
s1 = Index(2, "s1,a")
s2 = Index(2, "s2,b")

@show T2 = ITensor(s1, s2)

T2 = ITensor(s1, s2) = ITensor ord=2
Dim 1: (dim=2|id=788|"a,s1")
Dim 2: (dim=2|id=115|"b,s2")
NDTensors.EmptyStorage{NDTensors.EmptyNumber, NDTensors.Dense{NDTensors.EmptyNumber, Vector{NDTensors.EmptyNumber}}}
 2×2




ITensor ord=2 (dim=2|id=788|"a,s1") (dim=2|id=115|"b,s2")
NDTensors.EmptyStorage{NDTensors.EmptyNumber, NDTensors.Dense{NDTensors.EmptyNumber, Vector{NDTensors.EmptyNumber}}}

In [14]:
#1D3サイトの分配関数
T3 = randomITensor(s1, s2, s3)

# 各インデックスに値を指定（0始まりではなく 1始まり！）
val = T3[s1 => 1, s2 => 2, s3 => 1]

Z = val * val

println(Z)

1.1737927246921138


In [26]:
Lx, Ly = 3, 3  # 格子サイズ
sites = [Index(2, "s_$(x)_$(y)") for x in 1:Lx, y in 1:Ly]

3×3 Matrix{Index{Int64}}:
 (dim=2|id=830|"s_1_1")  (dim=2|id=363|"s_1_2")  (dim=2|id=29|"s_1_3")
 (dim=2|id=34|"s_2_1")   (dim=2|id=413|"s_2_2")  (dim=2|id=444|"s_2_3")
 (dim=2|id=365|"s_3_1")  (dim=2|id=862|"s_3_2")  (dim=2|id=315|"s_3_3")

In [27]:
function make_tensor(site::Index)
    # 簡単のために上下左右のボンド次元を2に固定
    up = Index(2, "u_$(site)")
    down = Index(2, "d_$(site)")
    left = Index(2, "l_$(site)")
    right = Index(2, "r_$(site)")
    return randomITensor(site, up, down, left, right)
end

T = [make_tensor(sites[x, y]) for x in 1:Lx, y in 1:Ly]

3×3 Matrix{ITensor}:
 ITensor ord=5
Dim 1: (dim=2|id=830|"s_1_1")
Dim 2: (dim=2|id=17|"u_(dim=2|id=830|")
Dim 3: (dim=2|id=367|"d_(dim=2|id=830|")
Dim 4: (dim=2|id=285|"l_(dim=2|id=830|")
Dim 5: (dim=2|id=525|"r_(dim=2|id=830|")
Dense{Float64, Vector{Float64}}
 2×2×2×2×2
[:, :, 1, 1, 1] =
 -0.51012  1.29651
  1.08189  1.66494

[:, :, 2, 1, 1] =
 -0.950124  -0.0308649
 -0.963813   0.475427

[:, :, 1, 2, 1] =
  1.88898   -1.01133
 -0.287457   0.972447

[:, :, 2, 2, 1] =
  0.242137  -0.277621
 -0.983578   0.752278

[:, :, 1, 1, 2] =
  1.26146   0.176534
 -0.141896  0.121593

[:, :, 2, 1, 2] =
  1.82926   1.11413
 -0.727649  0.954634

[:, :, 1, 2, 2] =
 0.848899  -1.07434
 0.509124   0.715596

[:, :, 2, 2, 2] =
 -0.437591  -1.96245
 -0.782389  -0.731854             …  ITensor ord=5
Dim 1: (dim=2|id=29|"s_1_3")
Dim 2: (dim=2|id=476|"u_(dim=2|id=29|"")
Dim 3: (dim=2|id=458|"d_(dim=2|id=29|"")
Dim 4: (dim=2|id=8|"l_(dim=2|id=29|"")
Dim 5: (dim=2|id=601|"r_(dim=2|id=29|"")
Dense{Float64, Vecto